# Lights, Camera, ReAction Functions

This notebook builds a full **sitcom script generation pipeline** powered by **ReAct-based agents** — a powerful framework that combines **reasoning** (thinking through a problem) with **acting** (taking structured steps).

We start by **generating a sitcom concept** from creative keywords, then **outlining** the pilot episode scene-by-scene.  
**Scene 1** is generated directly from the outline to establish the world and tone.  
After that, each new scene is **scripted, reviewed, and improved** using specialized **ReAct agents** — a **Character Agent**, **Comedy Agent**, and **Environment Agent** — that simulate a real sitcom writers' room.

---

 **Benefits of using ReAct agents**:

- **More structured and transparent thinking:** Agents reason step-by-step before making edits.
- **Dynamic adaptation:** Agents flexibly plan the next creative moves based on scene history.
- **Better long-term coherence:** Scenes evolve logically, with tracked character growth, running jokes, emotional arcs, and worldbuilding.

---

After generation, each scene is **summarized and stored in a vector database**, enabling fast retrieval of scene metadata for future story planning.  
By combining **structured agent workflows** and **retrieval-augmented memory**, we bring sitcom worlds to life — one coherent, character-driven scene at a time.


In [ ]:
!pip install datasets
!pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/utils") #Change to your path


In [ ]:
!ls "/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/utils"

character_agent.py     environment_helpers.py  script_review.py
character_helpers.py   outline_generation.py   text_utils.py
comedy_agent.py        __pycache__	       vector_db_utils.py
comedy_helpers.py      scene_planner_agent.py
environment_agents.py  screen_writing.py


In [ ]:
from character_agent import CharacterAgent
from comedy_agent import ComedicAgent
from environment_agents import EnvironmentReActAgent
from scene_planner_agent import ScenePlannerAgent

from text_utils import (
    display_markdown_output,
    extract_scene,
    extract_title,
)

from outline_generation import (
    generate_sitcom_pitch,
    generate_pilot_episode_outline
)


from script_review import (
    validate_episode_outline,
)

from screen_writing import (
    generate_scene_1_script,
    generate_scene
)


from vector_db_utils import (
    summarize_scene,
    add_scene_to_vector_db
)

In [ ]:
import numpy as np
import openai
from openai import OpenAI
from IPython.display import Markdown, display
import re
import time
from datetime import timedelta

In [ ]:
print(openai.__version__)

1.76.0


In [ ]:
from google.colab import userdata
import os

api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

### Vector Database

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Vector store initialization
dimension = embedding_model.get_sentence_embedding_dimension()
index = faiss.IndexFlatL2(dimension)

# To track metadata
vector_metadata = []


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Generating Pitch

- Generates pitch with key words

In [ ]:
keywords = [
    "urban locksmith shop",
    "ex-con protagonist",
    "estranged daughter",
    "quirky parole officer",
    "second chances",
    "buddy comedy",
    "New York City",
    "heartfelt absurdism",
    "odd couple dynamic"
]

**TODO**

Make sure the title is quotation marks

In [1]:
start_time = time.time()

# Generating Sitcom Pitch
sitcom_pitch = generate_sitcom_pitch(client, keywords)
display(Markdown(f"### Sitcom Concept:\n\n{sitcom_pitch}"))

end_time = time.time()
elapsed_time = end_time - start_time

print(" ")
print(f"Generation time: {elapsed_time:.2f} seconds")


NameError: name 'time' is not defined

### Episode Outline

In [ ]:
outline = generate_pilot_episode_outline(client, sitcom_pitch)
display(Markdown(f"### Sitcom Concept:\n\n{outline}"))

### Sitcom Concept:

Episode Concept: In the pilot episode, Charlie is released from prison and tries to reconnect with his estranged daughter, Lily, while also dealing with the comedic challenges of his parole officer, Gus, and the demands of setting up his locksmith shop.

Scene 1: "The Release" Charlie is discharged from prison, with a humorous mix-up involving his belongings.

Scene 2: "First Taste of Freedom" Charlie hilariously navigates the chaos of New York City, struggling with technology, modern fashion, and overpriced coffee.

Scene 3: "The Introduction" Charlie meets Officer Gus, his parole officer, in a comic sequence of misunderstandings and overly-enthusiastic advice.

Scene 4: "The Locksmith Shop" Charlie reopens his old locksmith shop, leading to comedic situations with rusty tools, forgotten skills, and oddball customers.

Scene 5: "The Surprise Visit" Gus unexpectedly drops by the shop, creating a funny scene as Charlie tries to hide the chaos of his struggling business.

Scene 6: "Letter to Lily" Charlie pens a heartfelt letter to his daughter Lily, trying to reconnect with her.

Scene 7: "Night in the Shop" Charlie has a strange encounter with a nocturnal customer, resulting in a hilarious misunderstanding.

Scene 8: "Return to Sender" Charlie's letter to Lily comes back as 'Return to Sender,' leading to a poignant moment of disappointment.

Scene 9: "Gus's Help" Gus offers to help Charlie find Lily, setting up a funny sequence as they search using outdated information.

Scene 10: "Awkward Stakeout" Charlie and Gus stake out Lily's last known address, leading to hilariously awkward interactions with the neighbors.

Scene 11: "The Wrong Lily" They mistakenly follow the wrong Lily, culminating in a comical blunder.

Scene 12: "Heart to Heart" Gus consoles a downbeat Charlie in a heartfelt conversation about second chances and fatherhood.

Scene 13: "Second Attempt" Inspired, Charlie writes another letter to Lily, this time dropping it off personally.

Scene 14: "Business Boom" Charlie's shop begins to pick up business, leading to a funny montage of him dealing with eccentric city dwellers.

Scene 15: "Gus's Inspection" Gus conducts a formal inspection of the shop, leading to a comedic sequence as Charlie tries to pass it off as a well-run establishment.

Scene 16: "Lily's Response" Charlie finds a response letter from Lily at his doorstep, leading to a touching moment.

Scene 17: "Mysterious Job" Charlie gets a locksmith call for a mysterious job, building suspense with humorous undertones.

Scene 18: "Unexpected Reunion" The client turns out to be Lily, leading to an emotional reunion layered with humor.

Scene 19: "Father-Daughter Reconnect" Charlie and Lily catch up in a touching conversation, interspersed with light-hearted banter.

Scene 20: "Closing Time" The episode closes with Charlie and Gus sharing a friendly drink, reflecting on the day's events with a blend of humor and sentimentality.

In [ ]:
validation = validate_episode_outline(client, sitcom_pitch, outline)
display(Markdown(f"### Coherence Evaluation:\n\n{validation}"))

### Coherence Evaluation:

Coherence: Yes

Reasoning:
- The outline is consistent with the premise of the sitcom. It follows the story of Charlie, an ex-con trying to reconnect with his estranged daughter while dealing with his parole officer and setting up his locksmith shop.
- The tone of the show is a balance of absurd comedy and heartfelt moments, which is reflected in the outline. Scenes like Charlie struggling with modern technology and the mix-up with his belongings provide the comedic elements, while his attempts to reconnect with his daughter and his conversations with Gus provide the emotional depth.
- The progression of the plot is logical. The episode starts with Charlie's release from prison and ends with him successfully reconnecting with his daughter and establishing his business, which aligns with the overall narrative arc of the sitcom.
- The character development is also coherent. Charlie grows from a confused ex-con to a successful business owner and a father trying to reconnect with his daughter. Gus's character, as an overly-enthusiastic and nurturing parole officer, is also well established through his interactions with Charlie.

## Scene 1 Generation

In [ ]:
scene_1_desc = extract_scene(outline, 1)
print(scene_1_desc)

Scene 1: "The Release" Charlie is discharged from prison, with a humorous mix-up involving his belongings.


In [ ]:
# Extracting title
sitcom_title = extract_title(sitcom_pitch)
print(sitcom_title)

Unlocked


In [ ]:
scene_1_script = generate_scene_1_script(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_1_desc,
    rag_context=None,
    line_target=(55, 75)
)

In [ ]:
print(scene_1_script)


INT. OAKVILLE PRISON - DAY

The scene opens with CHARLIE, a middle-aged man with an unkempt beard and a mischievous glint in his eyes, standing at a counter, GRUFF GUARD on the other side.

CHARLIE
(looking around)
Well, it's been real, but I won't say I'll miss this place.

GRUFF GUARD
(rolling eyes)
Just sign the release papers, Charlie.

Charlie signs with flourish. He then extends his hand eagerly towards a bag with his belongings.

CHARLIE
(looking at the bag)
I've been waiting for this moment.

GRUFF GUARD
(pushes the bag to Charlie)
Don't get too excited. It's just your old junk.

Charlie opens the bag with anticipation, then looks puzzled.

CHARLIE
(confused)
Uh, these aren't mine. 

He holds up a lacy black bra and a pair of pink fluffy handcuffs. 

[LAUGH TRACK]

GRUFF GUARD
(surprised, checks the bag)
Oh, my bad. This is from cell block D.

Charlie hands the bag back, trying not to laugh.

CHARLIE
(smirking)
Not my style, but I appreciate the thought.

[LAUGH TRACK]

Gruff 

### Adding Metadata to Vector Database

In [ ]:
scene_metadata = summarize_scene(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_1_script
)



In [ ]:
# Adds scene to the vector database
add_scene_to_vector_db(
    scene_metadata,
    full_script=scene_1_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)


In [ ]:
print("Total scenes stored in vector DB:", index.ntotal, "\n")

for i, meta in enumerate(vector_metadata):
    print(f"\nScene {i + 1}")
    print("Summary:", meta["summary"])
    print("Characters:", meta["characters"])
    print("Location:", meta["location"])
    print("Recurring Joke:", meta["recurring_joke"])
    print("Emotional Tone:", meta["emotional_tone"])


Total scenes stored in vector DB: 1 


Scene 1
Summary: The scene takes place in Oakville Prison, where Charlie, a middle-aged man with a mischievous personality, is being released. He signs his release papers and is handed a bag of his belongings. However, the bag initially given to him contains items that are not his, leading to a humorous exchange between him and the Gruff Guard. The correct bag is eventually given to Charlie, containing a rubber chicken, a clown nose, and a harmonica, which are more in line with his quirky character. Charlie exits the prison, wearing the clown nose, playing the harmonica, and carrying the rubber chicken, leaving the Gruff Guard shaking his head.
Characters: ['Charlie', 'Gruff Guard']
Location: Oakville Prison
Recurring Joke: None
Emotional Tone: Humorous


## Scene 2 Generation

In [ ]:
scene_2_desc = extract_scene(outline, 2)
print(scene_2_desc)

Scene 2: "First Taste of Freedom" Charlie hilariously navigates the chaos of New York City, struggling with technology, modern fashion, and overpriced coffee.


### Character Agent

In [ ]:
'''
import importlib
import character_agent
importlib.reload(character_agent)
from character_agent import CharacterAgent
'''

In [ ]:
# Initialize the agent
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=1  # optional — defaults to 1
)

character_histories, is_consistent, explanation, char_recommendations, thoughts = character_agent.run(
    scene_description=scene_2_desc,
    scene_number=2
)

print(f"Scene {2} — Consistency: {'✅ Consistent' if is_consistent else '❌ Inconsistent'}")
print("\nExplanation:\n", explanation)

print("\nInteraction Improvement Recommendations:\n", char_recommendations)

print("\nAgent's Internal Thoughts:")
for thought in thoughts:
    print("-", thought)


Scene 2 — Consistency: ✅ Consistent

Explanation:
 1. Consistency Verdict: Yes
2. Short Explanation Why: The planned scene is consistent with Charlie's established personality traits, emotional arc, and speaking style. His struggle with technology, modern fashion, and overpriced coffee aligns with his quirky and humorous personality. His nonchalant attitude towards these struggles is also in line with his speaking style. The scene continues his transition from prison to freedom, maintaining his light-hearted approach to life.

Interaction Improvement Recommendations:
 Interaction Recommendations:
1. Charlie should have a humorous exchange with a Barista at a modern coffee shop — In Scene 1, Charlie's quirky nonchalance and mischievous humor shine through. This interaction would allow Charlie's character to further display these traits as he baffles the Barista with his outdated coffee knowledge, maybe even attempting to pay with a two-dollar bill, causing an amusing confusion.
   
2. C

### Comedy Agent

In [ ]:
comedic_agent = ComedicAgent(client=client, vector_metadata=vector_metadata)

is_consistent, analysis_text, com_recommendations, thoughts = comedic_agent.run(
    scene_description=scene_2_desc,
    scene_number=2
)

print("Is Consistent:", is_consistent)
print("Analysis:\n", analysis_text)
print("Recommendations:\n", com_recommendations)
print("Internal Thoughts:\n", thoughts)

Is Consistent: True
Analysis:
 1. Detected Tone: Fish-out-of-water comedy
2. Consistency Verdict: Yes
3. Short Explanation: The comedic tone in the new scene is consistent with the previous scene. Both scenes utilize situational humor, with Charlie's quirky personality and unfamiliarity with his surroundings serving as the main source of comedy.
4. Overuse Check: None
5. Specific Suggestions if inconsistencies or overuse exist: N/A
Recommendations:
 Interaction Recommendations:
- [Suggestion 1]: Charlie, while attempting to use a smartphone for the first time, accidentally starts a Facebook live stream. Unaware of his global audience, he candidly critiques the city, its absurd fashion trends, and the baffling cost of coffee, generating a wave of likes, shares, and hilarious comments from viewers around the world.
- [Suggestion 2]: At a hipster cafe, Charlie orders a simple black coffee and is shocked when it costs $8. He humorously attempts to haggle the price down, mistaking the baris

### Enviroment Agent



In [ ]:
environment_agent = EnvironmentReActAgent(
    client=client,
    vector_metadata=vector_metadata,  # your list of prior scene metadata
    num_scenes=1  # or 2, 3, 5 — however many past scenes you want it to look at
)

In [ ]:
environment_analysis, transition_check, environment_details_suggestions, env_thoughts = environment_agent.run(
    scene_description=scene_2_desc,
    scene_number=2
)

In [ ]:
print(environment_details_suggestions)

Environment Details Suggestions:
- Suggestion 1: As the scene transitions to New York City, the characters could be seen struggling to hail a cab during rush hour, a common NYC struggle. This could add a humorous touch to the transition.
- Suggestion 2: The characters could walk past a trendy boutique with a window display showcasing outrageous, modern fashion styles accompanied by shockingly high price tags, reflecting the city's fashion scene.
- Suggestion 3: The characters could stop at a hipster coffee shop, where they're taken aback by the price of a small, artisanal latte. One of them could make a funny comment about how they could have bought a whole meal with that money back in prison.


### Scene Planner Agent

In [ ]:
scene_planner_agent = ScenePlannerAgent(client=client)

scene_2_plan = scene_planner_agent.plan_next_scene_explicit(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_details_suggestions=environment_details_suggestions,
    scene_number=2
)

In [ ]:
print(scene_2_plan)

Scene Plan:

Character Goals:
- Charlie has a humorous exchange with a Barista at a modern coffee shop, attempting to pay with a two-dollar bill, causing an amusing confusion.
- Charlie interacts with a young, tech-savvy character on the street, trying to understand modern technology, and baffling them with his outdated props like a rubber chicken or clown nose.

Comedic Goal:
- Charlie, while attempting to use a smartphone for the first time, accidentally starts a Facebook live stream. Unaware of his global audience, he candidly critiques the city, its absurd fashion trends, and the baffling cost of coffee.

Environment Detail:
- The characters walk past a trendy boutique with a window display showcasing outrageous, modern fashion styles accompanied by shockingly high price tags, reflecting the city's fashion scene.

Creative Suggestion:
- The next scene could take place at a bustling farmer's market. Charlie, still baffled by the high prices, tries to barter with the vendors using hi

### Generating Scene 2 Script

**Make sure the physical enviroment is conssitent over the whole scene**

**Define what a scene is to the agent**

In [ ]:
scene_2_script = generate_scene(
    client=client,
    scene_plan=scene_2_plan,
    scene_number= 2
)

print(scene_2_script)

INT. MODERN COFFEE SHOP - MORNING

CHARLIE (70s, old school, always seen with a rubber chicken and clown nose in his pockets) approaches the counter and places his order.

CHARLIE
(eyeing the menu)
I’ll have a regular coffee. How much is that?

BARISTA (20s, hipster)
That’s $5, sir. 

Charlie hands over a two-dollar bill. The Barista looks at it, confused. 

BARISTA (CONT’D)
Is this some kind of joke?

CHARLIE
(smiling)
No, it's a two-dollar bill. It's legal tender, son.

Barista shakes his head, amused.

CUT TO:

EXT. COFFEE SHOP - DAY

Charlie walks out of the coffee shop and bumps into a TECH SAVVY YOUTH (early 20s). 

TECH SAVVY YOUTH
(surprised, looking at his smartphone)
Dude! You just started a Facebook live stream!

Charlie looks at the smartphone, bewildered.

CHARLIE
What's a Facebook?

TECH SAVVY YOUTH
(laughs)
You're joking, right?

CHARLIE
No, seriously. What's a Facebook?

They walk past a trendy boutique with outrageous fashion styles displayed.

CHARLIE (CONT’D)
(pointi

### Adding Metadata to Vector Database

In [ ]:
scene_metadata_2 = summarize_scene(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_2_script
)


In [ ]:
# Adds scene to the vector database
add_scene_to_vector_db(
    scene_metadata_2,
    full_script=scene_2_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)


In [ ]:
print("Total scenes stored in vector DB:", index.ntotal, "\n")

for i, meta in enumerate(vector_metadata):
    print(f"\nScene {i + 1}")
    print("Summary:", meta["summary"])
    print("Characters:", meta["characters"])
    print("Location:", meta["location"])
    print("Recurring Joke:", meta["recurring_joke"])
    print("Emotional Tone:", meta["emotional_tone"])


Total scenes stored in vector DB: 2 


Scene 1
Summary: The scene takes place in Oakville Prison, where Charlie, a middle-aged man with a mischievous personality, is being released. He signs his release papers and is handed a bag of his belongings. However, the bag initially given to him contains items that are not his, leading to a humorous exchange between him and the Gruff Guard. The correct bag is eventually given to Charlie, containing a rubber chicken, a clown nose, and a harmonica, which are more in line with his quirky character. Charlie exits the prison, wearing the clown nose, playing the harmonica, and carrying the rubber chicken, leaving the Gruff Guard shaking his head.
Characters: ['Charlie', 'Gruff Guard']
Location: Oakville Prison
Recurring Joke: None
Emotional Tone: Humorous

Scene 2
Summary: In a modern coffee shop, Charlie, an old-school character in his 70s, orders a coffee and pays with a two-dollar bill, confusing the young barista. Outside, he bumps into a tech-s

## Scene 3 Generation

In [ ]:
scene_3_desc = extract_scene(outline, 3)
print(scene_3_desc)

Scene 3: "The Introduction" Charlie meets Officer Gus, his parole officer, in a comic sequence of misunderstandings and overly-enthusiastic advice.


### Character Agent

In [ ]:
# Initialize the agent
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=2
)

character_histories, is_consistent, explanation, char_recommendations, thoughts = character_agent.run(
    scene_description=scene_3_desc,
    scene_number=3
)

# Outputs
print(f"Scene {3} — Consistency: {'✅ Consistent' if is_consistent else '❌ Inconsistent'}")
print("\nExplanation:\n", explanation)

print("\nInteraction Improvement Recommendations:\n", char_recommendations)

print("\nAgent's Internal Thoughts:")
for thought in thoughts:
    print("-", thought)


Scene 3 — Consistency: ✅ Consistent

Explanation:
 1. Consistency Verdict: Yes
2. Short Explanation Why: Charlie's interaction with Officer Gus aligns with his established personality traits of being mischievous and quirky, as seen in his humorous exchanges in previous scenes. The comic sequence of misunderstandings also fits with Charlie's confusion with modern trends. Officer Gus's overly-enthusiastic advice is consistent with the brief character description provided. However, more information is needed to fully assess Officer Gus's consistency.

Interaction Improvement Recommendations:
 Interaction Recommendations:
1. Officer Gus, in his enthusiasm to guide Charlie, introduces him to a modern digital device, like a tablet or a smartphone, to help him navigate the modern world. Charlie, bewildered as usual, accidentally triggers a video call or stream, creating a comic sequence of misunderstandings. — This interaction fits because in Scene 2, Charlie accidentally starts a Facebook li

### Comedy Agent

In [ ]:
comedic_agent = ComedicAgent(client=client, vector_metadata=vector_metadata)

is_consistent, analysis_text, com_recommendations, thoughts = comedic_agent.run(
    scene_description=scene_3_desc,
    scene_number=3
)

print("Is Consistent:", is_consistent)
print("Analysis:\n", analysis_text)
print("Recommendations:\n", com_recommendations)
print("Internal Thoughts:\n", thoughts)

Is Consistent: True
Analysis:
 1. Detected Tone: Slapstick
2. Consistency Verdict: Yes
3. Short Explanation: The comedic tone in the new scene is consistent with the previous scenes. The humor continues to be derived from Charlie's quirky character and his interactions with others, leading to misunderstandings and humorous situations.
4. Overuse Check: None
5. Specific Suggestions if inconsistencies or overuse exist: N/A
Recommendations:
 Interaction Recommendations:
- [Suggestion 1]: As Charlie enters Officer Gus's office, he misinterprets "Take a seat" as an invitation to take the chair home. He starts to pick up the chair with a grateful smile, leading to a humorous confusion. This could establish Charlie as a literal-minded, overly-enthusiastic character.

- [Suggestion 2]: Officer Gus could have a peculiar habit of using police jargon in regular conversation. When he explains the terms of Charlie’s parole, he does so in such a convoluted way that Charlie just nods along, completel

### Enviroment Agent


In [ ]:
environment_agent = EnvironmentReActAgent(
    client=client,
    vector_metadata=vector_metadata,  # your list of prior scene metadata
    num_scenes=2  # or 2, 3, 5 — however many past scenes you want it to look at
)

In [ ]:
environment_analysis, transition_check, environment_details_suggestions, env_thoughts = environment_agent.run(
    scene_description=scene_3_desc,
    scene_number=3
)

In [ ]:
print(environment_details_suggestions)

Environment Details Suggestions:
- As Charlie enters the parole office, Officer Gus is absentmindedly playing with a rubber band ball, a collection of confiscated contraband items from parolees on his desk like a bizarre trophy collection. 
- The sound of a broken air conditioner that makes a comical 'sputter-clunk' noise every few minutes, causing Officer Gus to roll his eyes and mutter something about the budget cuts.
- The smell of stale coffee lingers in the air, mixed with the faintly metallic scent of old paper from the parole paperwork stacked haphazardly around the room.


### Scene Planner Agent

In [ ]:
scene_planner_agent = ScenePlannerAgent(client=client)

scene_3_plan = scene_planner_agent.plan_next_scene_explicit(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_details_suggestions=environment_details_suggestions,
    scene_number=3
)

In [ ]:
print(scene_3_plan)

Scene Plan:
Character Goals:
- Officer Gus introduces Charlie to a modern digital device, like a tablet or smartphone, with Charlie accidentally triggering a video call or stream.
- Charlie tries to pay Officer Gus with a two-dollar bill for his advice or services, leading to a humorous interaction where Officer Gus tries to refuse the payment.

Comedic Goal:
- As Charlie enters Officer Gus's office, he misinterprets "Take a seat" as an invitation to take the chair home and starts to pick up the chair with a grateful smile.

Environment Detail:
- As Charlie enters the parole office, Officer Gus is absentmindedly playing with a rubber band ball, a collection of confiscated contraband items from parolees on his desk like a bizarre trophy collection. 

Creative Suggestion:
- As Officer Gus tries to explain how to use the digital device to Charlie, Charlie interrupts him by pointing out the rubber band ball and asking if he could have one. This could lead to a funny back-and-forth as Gus t

### Generating Scene 3 Script

In [ ]:
scene_3_script = generate_scene(
    client=client,
    scene_plan=scene_3_plan,
    scene_number= 3
)

print(scene_3_script)

INT. PAROLE OFFICE - DAY

Officer GUS, a grizzled veteran with a soft spot for tech gadgets, is playing with a rubber band ball and scrolling through his tablet. CHARLIE, a naive ex-con with a heart of gold, ambles in, his eyes wide at the sight of the office.

GUS
(innocently)
Take a seat, Charlie.

Charlie beams, bends down, and starts to lift the chair.

GUS
(surprised)
What the...? No, Charlie! I meant sit down!

Charlie laughs sheepishly and sits down. Gus places his tablet in front of Charlie, who eyes it as if it's an alien object.

GUS
(excited)
This, Charlie, is a tablet. It's like a portable computer.

Charlie nods slowly, his eyes never leaving the tablet. He reaches out and accidentally triggers a video call. Suddenly, a WOMAN appears on screen.

WOMAN
(cheerily)
Oh hi, Gus! Who's your friend?

Gus sighs, grabbing the tablet and ending the call swiftly.

GUS
(flustered)
That was... never mind. You need to be careful, Charlie.

Charlie, still a little dazed, pulls out a two-

### Adding Metadata to Vector Database

In [ ]:
'''
MAKE FUNCTION!!!
'''

scene_metadata_3 = summarize_scene(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_3_script
)


# Adds scene to the vector database
add_scene_to_vector_db(
    scene_metadata_3,
    full_script=scene_3_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)


print("Total scenes stored in vector DB:", index.ntotal, "\n")

for i, meta in enumerate(vector_metadata):
    print(f"\nScene {i + 1}")
    print("Summary:", meta["summary"])
    print("Characters:", meta["characters"])
    print("Location:", meta["location"])
    print("Recurring Joke:", meta["recurring_joke"])
    print("Emotional Tone:", meta["emotional_tone"])


Total scenes stored in vector DB: 3 


Scene 1
Summary: The scene takes place in Oakville Prison, where Charlie, a middle-aged man with a mischievous personality, is being released. He signs his release papers and is handed a bag of his belongings. However, the bag initially given to him contains items that are not his, leading to a humorous exchange between him and the Gruff Guard. The correct bag is eventually given to Charlie, containing a rubber chicken, a clown nose, and a harmonica, which are more in line with his quirky character. Charlie exits the prison, wearing the clown nose, playing the harmonica, and carrying the rubber chicken, leaving the Gruff Guard shaking his head.
Characters: ['Charlie', 'Gruff Guard']
Location: Oakville Prison
Recurring Joke: None
Emotional Tone: Humorous

Scene 2
Summary: In a modern coffee shop, Charlie, an old-school character in his 70s, orders a coffee and pays with a two-dollar bill, confusing the young barista. Outside, he bumps into a tech-s

## Scene 4 Generation

In [ ]:
scene_4_desc = extract_scene(outline, 4)
print(scene_4_desc)

Scene 4: "The Locksmith Shop" Charlie reopens his old locksmith shop, leading to comedic situations with rusty tools, forgotten skills, and oddball customers.


### Character Agent

In [ ]:
# Initialize the agent
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

character_histories, is_consistent, explanation, char_recommendations, thoughts = character_agent.run(
    scene_description=scene_4_desc,
    scene_number=4
)

# Outputs
print(f"Scene {4} — Consistency: {'✅ Consistent' if is_consistent else '❌ Inconsistent'}")
print("\nExplanation:\n", explanation)

print("\nInteraction Improvement Recommendations:\n", char_recommendations)

print("\nAgent's Internal Thoughts:")
for thought in thoughts:
    print("-", thought)


Scene 4 — Consistency: ✅ Consistent

Explanation:
 1. Consistency Verdict: Yes
2. Short Explanation Why: Charlie's reopening of his old locksmith shop aligns with his old-school personality trait and his nostalgia for simpler times. His comedic interactions with rusty tools and forgotten skills are consistent with his quirky and mischievous traits, as well as his humorous speaking style. The introduction of oddball customers also provides an opportunity for Charlie to display his naivety, consistent with his previous interactions. There are no contradictions based on past scenes.

Interaction Improvement Recommendations:
 Interaction Recommendations:

1. Charlie has a humorous exchange with a customer who wants to pay with a credit card, but Charlie insists on accepting only cash, preferably two-dollar bills — This interaction fits because of Charlie's old-school preference for two-dollar bills, which was a running joke in Scene 2. Additionally, his lack of understanding of modern paym

### Comedy Agent

In [ ]:
comedic_agent = ComedicAgent(client=client, vector_metadata=vector_metadata)

is_consistent, analysis_text, com_recommendations, thoughts = comedic_agent.run(
    scene_description=scene_4_desc,
    scene_number=4
)

print("Is Consistent:", is_consistent)
print("Analysis:\n", analysis_text)
print("Recommendations:\n", com_recommendations)
print("Internal Thoughts:\n", thoughts)

Is Consistent: True
Analysis:
 1. Detected Tone: Slapstick
2. Consistency Verdict: Yes
3. Short Explanation: The comedic tone in the new scene is consistent with the previous scenes. Charlie's misadventures with rusty tools and forgotten skills, as well as his interactions with oddball customers, continue to highlight his quirky and mischievous personality, similar to his previous encounters with modern technology and trends.
4. Overuse Check: None
5. Specific Suggestions if inconsistencies or overuse exist: N/A
Recommendations:
 Interaction Recommendations:
- [Suggestion 1] Charlie tries to pick a lock for a demonstration and confidently assures a customer it's an "easy job". However, his skills are so rusty that he ends up jamming the lock instead. The customer then casually pops it open with a hairpin, saying, "Just like in the movies, huh?" 
- [Suggestion 2] A running gag could be that every time Charlie finds an old tool, he gets overly excited and nostalgic. He starts to share th

### Enviroment Agent


In [ ]:
environment_agent = EnvironmentReActAgent(
    client=client,
    vector_metadata=vector_metadata,  # your list of prior scene metadata
    num_scenes=3
)

In [ ]:
environment_analysis, transition_check, environment_details_suggestions, env_thoughts = environment_agent.run(
    scene_description=scene_4_desc,
    scene_number=4
)

In [ ]:
print(environment_details_suggestions)

Environment Details Suggestions:
- Suggestion 1: The locksmith is working on a particularly stubborn lock when the characters enter. He grunts and mutters under his breath, trying to remember a trick he hasn't used in years. The tension builds until he finally throws his hands up and declares, "Ah, got it!" only to reveal he's simply decided to break the lock open.
- Suggestion 2: An eccentric customer is seen in the background, trying to describe a very unusual key she needs duplicated. She uses wild gestures and strange, non-technical terms ("It's kind of like a squiggly line with a bumpy bit at the end!"). The locksmith looks bewildered, but is trying his best to understand and make the key.
- Suggestion 3: The shop has a distinct smell of metal and oil. There's the constant sound of clinking tools and the occasional whirr of a machine. The air is filled with fine dust particles that catch the light, creating a "snow globe" effect.


### Scene Planner Agent

In [ ]:
scene_planner_agent = ScenePlannerAgent(client=client)

scene_4_plan = scene_planner_agent.plan_next_scene_explicit(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_details_suggestions=environment_details_suggestions,
    scene_number=4
)

In [ ]:
print(scene_4_plan)

Scene Plan:
Character Goals:
- Charlie has a humorous exchange with a customer who wants to pay with a credit card, but Charlie insists on accepting only cash, preferably two-dollar bills.
- Charlie attempts to fix a lock using a rusty old tool, but ends up accidentally video calling Officer Gus on the shop's tablet. He then seeks Gus's advice on the lock problem while trying to figure out how to end the call.

Comedic Goal:
- Charlie tries to pick a lock for a demonstration and confidently assures a customer it's an "easy job". However, his skills are so rusty that he ends up jamming the lock instead. The customer then casually pops it open with a hairpin, saying, "Just like in the movies, huh?"

Environment Detail:
- The shop has a distinct smell of metal and oil. There's the constant sound of clinking tools and the occasional whirr of a machine. The air is filled with fine dust particles that catch the light, creating a "snow globe" effect.

Creative Suggestion:
- As Charlie is tryi

### Generating Scene 4 Script

In [ ]:
scene_4_script = generate_scene(
    client=client,
    scene_plan=scene_4_plan,
    scene_number= 4
)

print(scene_4_script)

INT. CHARLIE'S LOCKSMITH SHOP - DAY

We see CHARLIE, a charmingly gruff locksmith, haggling with a CUSTOMER who's trying to pay with a credit card.

CUSTOMER
(brandishing credit card)
I really don't carry cash, Charlie.

CHARLIE
(skeptically)
And I told ya, I only accept cash... preferably in two-dollar bills.

CUSTOMER
Who even has two-dollar bills these days?

CHARLIE
(winks)
Keeps the business interesting.

The customer shakes his head, amused, and leaves a cash tip on the counter. Charlie grins, then turns to a rusty lock on his workbench.

CHARLIE
(to himself)
Easy job, this.

He picks up a rusty tool and starts fiddling with the lock. Suddenly, the shop's tablet starts RINGING. Charlie accidentally answers the video call. It's OFFICER GUS.

CHARLIE
(startled)
Gus! 

OFFICER GUS
(squinting at screen)
Charlie? You know this isn't a phone call, right? 

CHARLIE
(panicking)
I'm trying to end this call, Gus!

Charlie continues to struggle with the tablet and the lock. Suddenly the Cus

## Scene 5 Generation

In [ ]:
scene_5_desc = extract_scene(outline, 5)
print(scene_5_desc)

Scene 5: "The Surprise Visit" Gus unexpectedly drops by the shop, creating a funny scene as Charlie tries to hide the chaos of his struggling business.


### Character Agent

In [ ]:
# Initialize the agent
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

character_histories, is_consistent, explanation, char_recommendations, thoughts = character_agent.run(
    scene_description=scene_5_desc,
    scene_number=5
)

# Outputs
print(f"Scene {5} — Consistency: {'✅ Consistent' if is_consistent else '❌ Inconsistent'}")
print("\nExplanation:\n", explanation)

print("\nInteraction Improvement Recommendations:\n", char_recommendations)

print("\nAgent's Internal Thoughts:")
for thought in thoughts:
    print("-", thought)


Scene 5 — Consistency: ✅ Consistent

Explanation:
 1. Consistency Verdict: Yes
2. Short Explanation Why: Gus's spontaneous visit aligns with his established personality trait of being spontaneous and perhaps a bit oblivious. Charlie's attempt to hide the chaos of his struggling business is consistent with his mischievous and quirky personality traits. The interaction between Gus and Charlie is logical based on their established relationship from the previous scenes. There are no contradictions identified based on past scenes.

Interaction Improvement Recommendations:
 Interaction Recommendations:
1. Charlie tries to pay Gus for his unexpected visit with a two-dollar bill, insisting it's a "tip for the service." Gus, confused, ends up accepting the bill in a humorous exchange. This interaction fits because Charlie has a habit of using two-dollar bills for transactions, which was established in Scene 2 and 3, creating confusion for those on the receiving end. 

2. Charlie misinterprets G

### Comedy Agent

In [ ]:
comedic_agent = ComedicAgent(client=client, vector_metadata=vector_metadata)

is_consistent, analysis_text, com_recommendations, thoughts = comedic_agent.run(
    scene_description=scene_5_desc,
    scene_number=5
)

print("Is Consistent:", is_consistent)
print("Analysis:\n", analysis_text)
print("Recommendations:\n", com_recommendations)
print("Internal Thoughts:\n", thoughts)

Is Consistent: True
Analysis:
 1. Detected Tone: Slapstick
2. Consistency Verdict: Yes
3. Short Explanation: The comedic tone in the new scene is consistent with the previous scenes. The humor continues to stem from Charlie's quirky character and his interactions with the more serious and modern characters, such as Gus.
4. Overuse Check: None
5. Specific Suggestions if inconsistencies or overuse exist: N/A
Recommendations:
 Interaction Recommendations:
- [Suggestion 1] As Gus enters the shop, Charlie could quickly shove a pile of overdue bills under the counter, accidentally knocking over a display of merchandise. The domino effect creates a funny disaster as he tries to clean up while entertaining Gus.
- [Suggestion 2] Charlie could pretend to converse with a non-existent customer on the phone to create the illusion of a bustling business. He could say things like, "No, I'm sorry, we are fully booked for the next two weeks" or "Yes, our premium service is in high demand." He could eve

### Enviroment Agent


In [ ]:
environment_agent = EnvironmentReActAgent(
    client=client,
    vector_metadata=vector_metadata,  # your list of prior scene metadata
    num_scenes=3
)

environment_analysis, transition_check, environment_details_suggestions, env_thoughts = environment_agent.run(
    scene_description=scene_5_desc,
    scene_number=5
)

print(environment_details_suggestions)

Environment Details Suggestions:
- As Gus and Charlie are dealing with a struggling business, there could be a humorous sign in the background that says "Buy 1, Get 10 Free!" to emphasize their desperation to sell products.
- The shop could feature a variety of odd, mismatched items (like a taxidermied raccoon, a broken disco ball, a half-eaten sandwich in a display case), highlighting the chaotic nature of their business.
- The shop could have a distinct smell of mixed scents - old books, motor oil, and freshly baked cookies - again reinforcing the eclectic and chaotic inventory of the shop.


### Scene Planner Agent

In [ ]:
scene_planner_agent = ScenePlannerAgent(client=client)

scene_5_plan = scene_planner_agent.plan_next_scene_explicit(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_details_suggestions=environment_details_suggestions,
    scene_number=5
)

print(scene_5_plan)

Scene Plan:
Character Goals:
- Charlie tries to pay Gus for his unexpected visit with a two-dollar bill, insisting it's a "tip for the service." Gus, confused, ends up accepting the bill in a humorous exchange.
- Charlie misinterprets Gus's visit as a tech support call and hands him the tablet he was struggling with in Scene 3.

Comedic Goal:
- Charlie could pretend to converse with a non-existent customer on the phone to create the illusion of a bustling business. He could say things like, "No, I'm sorry, we are fully booked for the next two weeks" or "Yes, our premium service is in high demand." He could even change his voice to pretend he's talking to a celebrity.

Environment Detail:
- The shop could feature a variety of odd, mismatched items (like a taxidermied raccoon, a broken disco ball, a half-eaten sandwich in a display case), highlighting the chaotic nature of their business.

Creative Suggestion:
- As Gus tries to fix Charlie's tablet, a customer walks into the shop, attrac

### Generating Scene 5 Script

In [ ]:
scene_5_script = generate_scene(
    client=client,
    scene_plan=scene_5_plan,
    scene_number= 5
)

print(scene_5_script)

INT. CHARLIE'S ODDITIES - DAY

Charlie, a middle-aged man with an eccentric personality, is sitting behind the counter, struggling with a tablet. Gus, a slightly bewildered but patient man, walks into the shop.

CHARLIE
(Without looking up)
Welcome to Charlie's Oddities, where the strange becomes familiar.

Gus looks around at the bizarre items, his eyebrows raised.

GUS
I can see that...

Charlie finally looks up, startled. He quickly reaches into his pocket, pulls out a two-dollar bill, and hands it to Gus.

CHARLIE
For your service. A tip, my good man.

GUS
(Confused)
Charlie, I'm just here because you called...

CHARLIE
(Interrupting)
And that's why I'm tipping you.

Gus, bemused, takes the two-dollar bill. Charlie hands him the tablet.

CHARLIE
And while you're here, Gus, I've been wrestling with this gizmo since scene 3.

Just then, Charlie's phone RINGS. He answers it with flair, speaking loudly enough for Gus and the whole shop to hear.

CHARLIE
(On the phone)
Charlie's Odditie

## Combining Scripts



In [ ]:
# List your scene scripts in order
scene_scripts = [
    scene_1_script,
    scene_2_script,
    scene_3_script,
    scene_4_script,
    scene_5_script
]

# Combine with optional scene headers
full_episode_script = "\n\n".join([
    f"### Scene {i+1} ###\n{script.strip()}"
    for i, script in enumerate(scene_scripts)
])

print(full_episode_script)

### Scene 1 ###
INT. OAKVILLE PRISON - DAY

The scene opens with CHARLIE, a middle-aged man with an unkempt beard and a mischievous glint in his eyes, standing at a counter, GRUFF GUARD on the other side.

CHARLIE
(looking around)
Well, it's been real, but I won't say I'll miss this place.

GRUFF GUARD
(rolling eyes)
Just sign the release papers, Charlie.

Charlie signs with flourish. He then extends his hand eagerly towards a bag with his belongings.

CHARLIE
(looking at the bag)
I've been waiting for this moment.

GRUFF GUARD
(pushes the bag to Charlie)
Don't get too excited. It's just your old junk.

Charlie opens the bag with anticipation, then looks puzzled.

CHARLIE
(confused)
Uh, these aren't mine. 

He holds up a lacy black bra and a pair of pink fluffy handcuffs. 

[LAUGH TRACK]

GRUFF GUARD
(surprised, checks the bag)
Oh, my bad. This is from cell block D.

Charlie hands the bag back, trying not to laugh.

CHARLIE
(smirking)
Not my style, but I appreciate the thought.

[LAUGH